# 01 - Bronze Ingestion

In [ ]:
import os

# Config
os.environ["AZURE_TENANT_ID"] = "id"
os.environ["AZURE_CLIENT_ID"] = "id"
os.environ["AZURE_CLIENT_SECRET"] = "id"

os.environ["STEAM__API_KEY"] = "id"
os.environ["FABRIC__WORKSPACE_ID"] = "id"
os.environ["FABRIC__BRONZE_LAKEHOUSE_ID"] = "id"
os.environ["FABRIC__SILVER_LAKEHOUSE_ID"] = "id"
os.environ["FABRIC__GOLD_LAKEHOUSE_ID"] = "id"

import sys
sys.path.insert(0, "/lakehouse/default/Files")

from src.steam_analytics.config import Settings
settings = Settings()

BRONZE_PATH = settings.fabric.bronze_abfss_path
CATALOG_PATH = f"{BRONZE_PATH}/Tables/game_catalog"

print(f"Bronze Path: {BRONZE_PATH}")
print(f"Catalog Path: {CATALOG_PATH}")

In [ ]:
from delta.tables import DeltaTable
from pyspark.sql.utils import AnalysisException

# Si ya tienes las variables cargadas, ignora estas líneas de config
# from src.steam_analytics.config import Settings
# settings = Settings()
# BRONZE_PATH = settings.fabric.bronze_abfss_path
# ... etc

print("=" * 60)
print("🧹 INICIANDO VACIADO DE TABLAS (MANTENIENDO ESTRUCTURA)")
print("=" * 60)

# Lista de tablas a vaciar
tables_to_truncate = [
    f"{BRONZE_PATH}/Tables/game_catalog",
]

for table_path in tables_to_truncate:
    try:
        # 1. Conectamos con la tabla Delta en esa ruta
        delta_table = DeltaTable.forPath(spark, table_path)
        
        # 2. Ejecutamos DELETE sin condiciones (Borra todo)
        delta_table.delete()
        
        # 3. Verificamos que quedó vacía
        count = delta_table.toDF().count()
        print(f"✅ {table_path.split('/')[-1]}: Vaciada correctamente. Filas restantes: {count}")
        
    except AnalysisException:
        # Esto pasa si la tabla no existe todavía
        print(f"⚠️ {table_path.split('/')[-1]}: No existe o no es una tabla Delta válida.")
    except Exception as e:
        print(f"❌ Error en {table_path.split('/')[-1]}: {str(e)}")

print("\n✨ Proceso terminado. Las tablas están vacías y listas.")

In [ ]:
from pyspark.sql import functions as F
from datetime import datetime
from src.steam_analytics.catalog import GameCatalogManager, SyncPriority

print("=" * 60)
print("LOADING CATALOG")
print("=" * 60)

# Load catalog
catalog_df = spark.read.format("delta").load(CATALOG_PATH)
total_in_catalog = catalog_df.count()
print(f"Total games in catalog: {total_in_catalog:,}")

# Determine priorities based on day of week
manager = GameCatalogManager()
day_of_week = datetime.utcnow().weekday()
day_names = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

priorities = manager.get_sync_schedule(day_of_week)
priority_values = [p.value for p in priorities]

print(f"\nToday: {day_names[day_of_week]}")
print(f"Priorities to sync: {priority_values}")

# Filter catalog by priority
apps_to_sync_df = catalog_df.filter(
    F.col("priority").isin(priority_values)
)

# Get app IDs
APP_IDS = [row.app_id for row in apps_to_sync_df.select("app_id").collect()]

print(f"\nGames to sync today: {len(APP_IDS):,}")

# Show breakdown
print("\nBreakdown by priority:")
apps_to_sync_df.groupBy("priority").count().orderBy("priority").show()

# Estimate time
estimate = manager.estimate_sync_time(len(APP_IDS))
print(f"Estimated time: ~{estimate['estimated_minutes']:.0f} min")

In [ ]:
import asyncio
from src.steam_analytics.ingestion.orchestrator import IngestionOrchestrator, OutputTarget

print("=" * 60)
print("RUNNING INGESTION")
print("=" * 60)
print(f"Games to process: {len(APP_IDS):,}")

orchestrator = IngestionOrchestrator(target=OutputTarget.ONELAKE)
result = await orchestrator.run(APP_IDS)

In [ ]:
from delta.tables import DeltaTable
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType

print("=" * 60)
print("UPDATING CATALOG (DELTA MERGE)")
print("=" * 60)

# Get successfully synced app IDs
synced_ids = APP_IDS  # O usa result.processed_ids si tu orquestador lo devuelve

if synced_ids:
    now = datetime.utcnow()
    
    # 1. Crear un DataFrame pequeño SOLO con las actualizaciones (Updates)
    # Esto es mucho más rápido que leer todo el catálogo de 150k filas
    updates_data = [{"app_id": app_id, "last_synced_at": now} for app_id in synced_ids]
    
    updates_schema = StructType([
        StructField("app_id", IntegerType(), False),
        StructField("last_synced_at", TimestampType(), True)
    ])
    
    updates_df = spark.createDataFrame(updates_data, schema=updates_schema)
    
    # 2. Instanciar la tabla Delta destino
    target_table = DeltaTable.forPath(spark, CATALOG_PATH)
    
    # 3. Ejecutar el MERGE
    (target_table.alias("target")
        .merge(
            updates_df.alias("source"),
            "target.app_id = source.app_id"
        )
        .whenMatchedUpdate(set={
            "last_synced_at": "source.last_synced_at"
        })
        .execute()
    )
    
    print(f"✅ Efficiently merged updates for {len(synced_ids):,} games")

else:
    print("No updates to merge.")

## Results

In [ ]:
from notebookutils import mssparkutils
import json

print("=" * 60)
print("BRONZE INGESTION COMPLETE")
print("=" * 60)
print(f"Run ID:         {result.run_id}")
print(f"Duration:       {result.duration_seconds:.2f}s")
print(f"Batches:        {len(result.batches_written)}")
print(f"Success Rate:   {result.success_rate}%")
print(f"Games Synced:   {len(APP_IDS):,}")

if result.errors:
    print(f"Errors:         {len(result.errors)}")
    for err in result.errors[:5]:
        print(f"  - {err}")

# Exit with status
mssparkutils.notebook.exit(json.dumps({
    "status": "success" if result.success_rate >= 80.0 else "partial",
    "run_id": str(result.run_id),
    "batches_written": len(result.batches_written),
    "success_rate": result.success_rate,
    "games_synced": len(APP_IDS),
}, default=str))